DETR
====

**End-to-End Object Detection with Transformers**

 * Paper: https://arxiv.org/abs/2005.12872

![DETR Overview](../assets/detr_overview.png)

```bash
pip install torch torchvision
pip install transformers
pip install timm
pip install supervision
```